In [46]:
import pandas as pd
import numpy as np
# https://pandas.pydata.org/pandas-docs/stable/generated/pandas.DataFrame.from_csv.html
event_key = '2017txlu'
matches_file = event_key + '_matches.csv'
stats_file = event_key + '_stats.csv'
experience_file = event_key + '_rookieyear.csv'
matches = pd.DataFrame.from_csv(matches_file)
stats = pd.DataFrame.from_csv(stats_file)
experience = pd.DataFrame.from_csv(experience_file)
teams_df = pd.merge(experience, stats, on = "team")

In [47]:
experience.head()

,team,experience
0,frc1164,14
1,frc118,20
2,frc1477,13
3,frc159,19
4,frc1817,11


In [3]:
teams = experience["team"].tolist()
teams.append("score")
df_teams = pd.DataFrame(columns = teams)
qm = matches.loc[matches["level"] == "qm"]

In [4]:
# https://erikrood.com/Python_References/rows_cols_python.html
# https://pandas.pydata.org/pandas-docs/stable/generated/pandas.DataFrame.reset_index.html
# http://book.pythontips.com/en/latest/enumerate.html
# # https://stackoverflow.com/questions/22963263/creating-a-zero-filled-pandas-data-frame
# https://www.chiefdelphi.com/media/papers/2174
# https://www.chiefdelphi.com/forums/showthread.php?t=101390
# https://blog.thebluealliance.com/2017/10/05/the-math-behind-opr-an-introduction/
m = np.zeros((len(teams), len(teams)))
for counter, team in enumerate(teams):
    # create dataframe of only games played by one team
    blue_team = qm.loc[(qm["b1"] == team) | (qm["b2"] == team) | (qm["b3"] == team)]
    red_team = qm.loc[(qm["r1"] == team) | (qm["r2"] == team) | (qm["r3"] == team)]
    indiv_team = pd.concat([blue_team, red_team])
    indiv_team.reset_index(inplace = True)
    matrix = pd.DataFrame(0, index = np.arange(len(indiv_team)), columns = teams)
    # update dataframe with scores and who played in the match
    for index, row in indiv_team.iterrows():
        b1 = row["b1"]
        b2 = row["b2"]
        b3 = row["b3"]
        blue = []
        blue.extend([b1, b2, b3])
        bscore = row["bscore"]
        r1 = row["r1"]
        r2 = row["r2"]
        r3 = row["r3"]
        red = []
        red.extend([r1, r2, r3])
        rscore = row["rscore"]
        if team in blue:
            matrix.loc[index, b1] = 1
            matrix.loc[index, b2] = 1
            matrix.loc[index, b3] = 1
            matrix.loc[index, "score"] = bscore
        else:
            matrix.loc[index, r1] = 1
            matrix.loc[index, r2] = 1
            matrix.loc[index, r3] = 1
            matrix.loc[index, "score"] = rscore
    new_row = matrix.sum(axis = 0)
    m[counter, :] = new_row
m

array([[  1.10000000e+01,   1.00000000e+00,   1.00000000e+00, ...,
          1.00000000e+00,   0.00000000e+00,   1.82300000e+03],
       [  1.00000000e+00,   1.10000000e+01,   0.00000000e+00, ...,
          0.00000000e+00,   0.00000000e+00,   2.28000000e+03],
       [  1.00000000e+00,   0.00000000e+00,   1.10000000e+01, ...,
          1.00000000e+00,   1.00000000e+00,   2.51700000e+03],
       ..., 
       [  1.00000000e+00,   0.00000000e+00,   1.00000000e+00, ...,
          1.10000000e+01,   1.00000000e+00,   1.63200000e+03],
       [  0.00000000e+00,   0.00000000e+00,   1.00000000e+00, ...,
          1.00000000e+00,   1.10000000e+01,   1.93000000e+03],
       [  0.00000000e+00,   0.00000000e+00,   0.00000000e+00, ...,
          0.00000000e+00,   0.00000000e+00,   0.00000000e+00]])

In [42]:
# grab info that is teams only
M = m.copy()
M = M[:, 0:len(teams)-1]
M

array([[ 11.,   1.,   1., ...,   0.,   1.,   0.],
       [  1.,  11.,   0., ...,   0.,   0.,   0.],
       [  1.,   0.,  11., ...,   1.,   1.,   1.],
       ..., 
       [  1.,   0.,   1., ...,   1.,  11.,   1.],
       [  0.,   0.,   1., ...,   1.,   1.,  11.],
       [  0.,   0.,   0., ...,   0.,   0.,   0.]])

In [43]:
# grab info that is scores only
s = m.copy()
s = s[:, -1]
s

array([ 1823.,  2280.,  2517.,  1909.,  2045.,  2227.,  2240.,  1925.,
        2041.,  1601.,  1943.,  2591.,  1850.,  1742.,  1608.,  1562.,
        2521.,  1455.,  1941.,  1799.,  1858.,  1482.,     0.,  1278.,
        1992.,  1678.,  1674.,  2122.,  1807.,  1186.,  1785.,  1476.,
        1810.,  1694.,  2254.,  1501.,  1497.,  1288.,  1745.,  1844.,
        1406.,  1519.,  1626.,  2133.,  1632.,  1930.,     0.])

In [44]:
# https://docs.scipy.org/doc/numpy-dev/user/numpy-for-matlab-users.html
# https://docs.scipy.org/doc/numpy-1.13.0/reference/generated/numpy.linalg.lstsq.html
OPR = np.linalg.lstsq(M,s)[0]

In [49]:
OPR[2]

118.23478943973753

In [28]:
len(teams)


47

In [66]:
m = np.zeros((len(teams), len(teams)))
for counter, team in enumerate(teams):
    # create dataframe of only games played by one team
    blue_team = qm.loc[(qm["b1"] == team) | (qm["b2"] == team) | (qm["b3"] == team)]
    red_team = qm.loc[(qm["r1"] == team) | (qm["r2"] == team) | (qm["r3"] == team)]
    indiv_team = pd.concat([blue_team, red_team])
    indiv_team.reset_index(inplace = True)
    matrix = pd.DataFrame(0, index = np.arange(len(indiv_team)), columns = teams)
    # update dataframe with scores and who played in the match
    for index, row in indiv_team.iterrows():
        b1 = row["b1"]
        b2 = row["b2"]
        b3 = row["b3"]
        blue = []
        blue.extend([b1, b2, b3])
        bscore = row["bscore"]
        r1 = row["r1"]
        r2 = row["r2"]
        r3 = row["r3"]
        red = []
        red.extend([r1, r2, r3])
        rscore = row["rscore"]
        if team in blue:
            matrix.loc[index, b1] = 1
            matrix.loc[index, b2] = 1
            matrix.loc[index, b3] = 1
            matrix.loc[index, "score"] = rscore
        else:
            matrix.loc[index, r1] = 1
            matrix.loc[index, r2] = 1
            matrix.loc[index, r3] = 1
            matrix.loc[index, "score"] = bscore
    new_row = matrix.sum(axis = 0)
    m[counter, :] = new_row

In [67]:
M = m.copy()
M = M[:, 0:len(teams)-1]
s = m.copy()
s = s[:, -1]
DPR = np.linalg.lstsq(M,s)[0]

In [68]:
df_OPR = pd.DataFrame(columns = ["team", "OPR", "DPR", "CCWM"])
for counter, team in enumerate(teams):
    if team != "score":
        df_OPR = df_OPR.append({"team": team, "OPR": OPR[counter], "DPR": DPR[counter]}, ignore_index = True)
df_OPR.head()

,team,OPR,DPR,CCWM
0,frc1164,41.842873,43.147364,NaN
1,frc118,109.437287,49.938890,NaN
2,frc1477,118.234789,12.249579,NaN
3,frc159,51.352990,44.305976,NaN
4,frc1817,78.688407,1.981109,NaN


In [69]:
def ccwm(row):
    return row["OPR"] - row["DPR"]
        

In [70]:
df_OPR["CCWM"] = df_OPR.apply(ccwm, axis = 1)
df_OPR.head()

,team,OPR,DPR,CCWM
0,frc1164,41.842873,43.147364,-1.304491
1,frc118,109.437287,49.938890,59.498398
2,frc1477,118.234789,12.249579,105.985210
3,frc159,51.352990,44.305976,7.047014
4,frc1817,78.688407,1.981109,76.707298


In [71]:
name = event_key + '_opr.csv'
df_OPR.to_csv(name)